##### Credits to Ming Wang for sharing the Jupyter notebook for the integration.

###### This notebook requires a .graphml file generated from FBMN. Once you run the job, using the GNPSexport files, save the graphml file under the directory results/GNPSexport and run the following cells to add the Sirius and CSI information to the cytoscape file (only compatible for a single file ATM):

In [1]:
# Integrating into Graphml
import requests
import pandas as pd
import networkx as nx
import glob
import os
import sys
from pyopenms import *
from pyteomics import mgf, auxiliary

Determination of memory status is not supported on this 
 platform, measuring for memoryleaks will never fail


Add the SCAN number from the MGF file in the SIRIUS merged library.

In [8]:
path= os.path.join("results", "GNPSexport", "MSMS.mgf")
Matrix= pd.read_csv(os.path.join("results", "features", "FeatureMatrix.tsv"), sep="\t")
Matrix["id"]= Matrix["id"].apply(lambda x: '%.0f' % x)
Matrix

,id,charge,RT,mz,quality,Pentamycin_std.mzML,Leupeptin_std.mzML,feature_ids
0,1028421331611893248,1,820.850089,584.437535,0.000363,506658.6,3.981326e+05,"[2270960838316941815, 701982464137687754]"
1,17890036615095214080,1,859.153349,598.453111,0.000496,692031.7,5.790350e+05,"[4320835582941733035, 11035819080249078724]"
2,400686241134609600,1,38.341091,239.106106,0.036639,52432290.0,1.074500e+08,"[5982674884583807406, 4420971766091018177]"
3,15927949919496843264,1,333.307483,185.114834,0.001431,2156296.0,2.314913e+06,"[5998372309151319305, 432537723525269015]"
4,12203235798799403008,1,585.663306,183.080447,0.000760,1128897.0,1.197214e+06,"[6050553011821882372, 15143896006720662432]"
...,...,...,...,...,...,...,...,...
173,17305628292915539968,2,333.216860,418.298128,0.000780,0.0,1.903450e+07,[17212658288873641448]
174,17070012996113844224,1,319.982463,367.270594,0.000038,0.0,1.307579e+06,[17763073354530953559]
175,4091971196934941184,1,298.998984,427.303201,0.369464,0.0,1.169240e+10,[17826928278156545246]
176,17467807095536789504,1,298.998984,254.197539,0.001173,0.0,4.224071e+07,[17950575855496761224]


In [39]:
file= mgf.MGF(source=path, use_header=True, convert_arrays=2, read_charges=True, read_ions=False, dtype=None, encoding=None)
for spectrum in file:
    print(spectrum)

{'params': {'output': 'most_intense', 'scans': '1', 'feature_id': 'e_11445279324287385245', 'mslevel': '2', 'charge': [1], 'pepmass': (262.97224786300535, None), 'file_index': '156', 'rtinseconds': 63.4662936841672}, 'm/z array': array([ 55.2152,  64.2597,  75.5506,  76.1114,  77.1874,  78.9493,
        87.1658,  96.9597,  97.0835, 101.496 , 114.9705, 127.6873,
       143.2919, 174.9737, 184.959 , 202.9692, 207.2606, 221.9924,
       272.8652]), 'intensity array': array([2455.4438, 2365.2688, 2432.4114, 2289.4143, 2755.2231, 2552.7761,
       2759.219 , 2935.8779, 2642.4167, 2458.6484, 4286.5234, 2708.2249,
       2711.9934, 3412.5779, 3556.7761, 4232.8052, 2901.5886, 2851.3132,
       2818.6553]), 'charge array': masked_array(data=[--, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --, --],
             mask=[ True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
           

In [5]:
# Adding sirius information
input_formulas= glob.glob(os.path.join("results", "Sirius", "formulas_*.tsv"))
DF_SIRIUS = pd.DataFrame()
list_of_df=[]
for tsv in input_formulas:
    df= pd.read_csv(tsv, sep="\t", index_col="Unnamed: 0")
    s= df["opt_global_rank"]
    pd.to_numeric(s)
    df= df.loc[df["opt_global_rank"]==1]
    df=df.reset_index()
    list_of_df.append(df)
DF_SIRIUS= pd.concat(list_of_df,ignore_index=True)
DF_SIRIUS= DF_SIRIUS.drop(columns="index")
DF_SIRIUS["opt_global_featureId"]= DF_SIRIUS["opt_global_featureId"].str.replace(r"id_", "")
DF_SIRIUS

,chemical_formula,exp_mass_to_charge,retention_time,SiriusScore,TreeScore,IsotopeScore,opt_global_adduct,opt_gobal_precursorFormula,opt_global_rank,opt_global_explainedPeaks,opt_global_explainedIntensity,opt_global_median_mass_error_fragment_peaks_ppm,opt_global_median_absolute_mass_error_fragment_peaks_ppm,opt_global_mass_error_precursor_ppm,opt_global_compoundId,opt_global_compoundScanNumber,opt_global_featureId,opt_global_native_id
0,C14H15N,198.127724,365.137542,35.157103,31.430176,3.726927,[M + H]+,C14H15N,1,7,0.889702,-1.881193,1.881193,-0.011519,2957,2958,id_8753035862846496565,controllerType=0 controllerNumber=1 scan=2958|...
1,C14H17NO,198.127724,365.137542,35.157103,31.430176,3.726927,[M - H2O + H]+,C14H15N,1,8,0.889702,-1.881193,1.881193,-90903.820195,2957,2958,id_8753035862846496565,controllerType=0 controllerNumber=1 scan=2958|...
2,C13H22O2,228.195690,475.173607,105.151326,101.471077,3.680250,[M + H3N + H]+,C13H25NO2,1,25,0.918288,-2.069488,2.214486,-0.506414,4003,4004,id_13199906195617743957,controllerType=0 controllerNumber=1 scan=4004
3,C13H25NO2,228.195690,475.173607,105.151326,101.471077,3.680250,[M + H]+,C13H25NO2,1,25,0.918288,-2.069488,2.214486,-0.506414,4003,4004,id_13199906195617743957,controllerType=0 controllerNumber=1 scan=4004
4,C13H27NO3,228.195690,475.173607,105.151326,101.471077,3.680250,[M - H2O + H]+,C13H25NO2,1,26,0.918288,-2.069488,2.214486,-78926.469869,4003,4004,id_13199906195617743957,controllerType=0 controllerNumber=1 scan=4004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,C14H17NO,198.127725,364.559057,31.035939,27.398263,3.637676,[M - H2O + H]+,C14H15N,1,6,0.898811,-0.946085,0.946085,-90903.815402,2591,2592,id_8238977793258643426,controllerType=0 controllerNumber=1 scan=2592
117,C13H25NO2,228.195649,474.566056,149.370470,145.716595,3.653875,[M + H]+,C13H25NO2,1,35,0.943079,-2.574929,2.574929,-0.685811,3385,3386,id_1202172533432230144,controllerType=0 controllerNumber=1 scan=3386
118,C13H27NO3,228.195649,474.566056,149.370470,145.716595,3.653875,[M - H2O + H]+,C13H25NO2,1,36,0.943079,-2.574929,2.574929,-78926.663424,3385,3386,id_1202172533432230144,controllerType=0 controllerNumber=1 scan=3386
119,C13H22O2,228.195649,474.566056,143.864060,140.210185,3.653875,[M + H3N + H]+,C13H25NO2,1,34,0.936937,-2.576616,2.576616,-0.685811,3385,3386,id_1202172533432230144,controllerType=0 controllerNumber=1 scan=3386


In [2]:
file_list= glob.glob(os.path.join("results", "GNPSexport", "*.graphml"))
for file in file_list:
    G = nx.read_graphml(file)

    for result in DF_SIRIUS.to_dict(orient="records"):
        scan = str(result["opt_global_compoundScanNumber"])
        if scan in G:
            G.nodes[scan]["sirius:molecularFormula"] = result["chemical_formula"]
            G.nodes[scan]["sirius:adduct"] = result["opt_global_adduct"]
            G.nodes[scan]["sirius:TreeScore"] = result["TreeScore"]
            G.nodes[scan]["sirius:IsotopeScore"] = result["IsotopeScore"]
            G.nodes[scan]["sirius:explainedPeaks"] = result["opt_global_explainedPeaks"]
            G.nodes[scan]["sirius:explainedIntensity"] = result["opt_global_explainedIntensity"]
            G.nodes[scan]["sirius:explainedPeaks"] = result["opt_global_explainedPeaks"]

    input_structures= glob.glob(os.path.join("results", "Sirius", "structures_*.tsv"))
    DF_CSI = pd.DataFrame()
    list_of_df=[]
    for tsv in input_structures:
        df= pd.read_csv(tsv, sep="\t", index_col="Unnamed: 0")
        s= df["opt_global_rank"]
        pd.to_numeric(s)
        df= df.loc[df["opt_global_rank"]==1]
        df=df.reset_index()
        list_of_df.append(df)
    DF_CSI= pd.concat(list_of_df,ignore_index=True)
    DF_CSI= DF_CSI.drop(columns="index")

    # Adding CSI:FingerID information
    for result in DF_CSI.to_dict(orient="records"):
        scan = str(result["opt_global_compoundScanNumber"])
        if scan in G:
            G.nodes[scan]["csifingerid:smiles"] = result["smiles"]
            G.nodes[scan]["csifingerid:Confidence_Score"] = result["best_search_engine_score[1]"]
            G.nodes[scan]["csifingerid:dbflags"] = result["opt_global_dbflags"]

    nx.write_graphml(G, os.path.join("results", "GNPSexport", "fbmn_sirius.graphml"))